In [9]:
!pip install rank_bm25
!git clone https://github.com/cr-nlp/project1-2023.git

fatal: destination path 'project1-2023' already exists and is not an empty directory.


In [10]:
import urllib.request as re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
import pandas as pd
from gensim.models import word2vec

In [11]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
def loadNFCorpus():
	dir = "./project1-2023/"
	filename = dir +"dev.docs"

	dicDoc={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		#print(tabLine)
		key = tabLine[0]
		value = tabLine[1]
		#print(value)
		dicDoc[key] = value
	filename = dir + "dev.all.queries"
	dicReq={}
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.split('\t')
		key = tabLine[0]
		value = tabLine[1]
		dicReq[key] = value
	filename = dir + "dev.2-1-0.qrel"
	dicReqDoc=defaultdict(dict)
	with open(filename) as file:
		lines = file.readlines()
	for line in lines:
		tabLine = line.strip().split('\t')
		req = tabLine[0]
		doc = tabLine[2]
		score = int(tabLine[3])
		dicReqDoc[req][doc]=score

	return dicDoc, dicReq, dicReqDoc

In [13]:
def text2TokenList(text):
	stopword = stopwords.words('english')
	#print("LEN DE STOPWORD=",len(stopword))
	word_tokens = word_tokenize(text.lower())
	word_tokens_without_stops = [word for word in word_tokens if word not in stopword and len(word)>2]
	return word_tokens_without_stops

In [14]:
def modelWord2vec(corpus):
    model = word2vec.Word2Vec(sentences=corpus, vector_size=1000, window=5, min_count=1, workers=4)
    corpus_embeddings = []
    for corp in corpus:
        embeddings = [model.wv[word] for word in corp if word in model.wv]
        corpus_embeddings.append(sum(embeddings) / len(embeddings))
    return corpus_embeddings

In [15]:
def run_bm25_only(startDoc,endDoc):

    dicDoc, dicReq, dicReqDoc = loadNFCorpus()
    #print(dicReqDoc)

    docsToKeep=[]
    reqsToKeep=[]
    dicReqDocToKeep=defaultdict(dict)

    #150
    ndcgTop=20
    print("ndcgTop=",ndcgTop,"nbDocsToKeep=",endDoc - startDoc)


    #Construction du dictionnaire de Req à garder
    i=startDoc
    for reqId in dicReqDoc:
        if i > (endDoc - startDoc) :  #nbDocsToKeep:
            break
        for docId in dicReqDoc[reqId]:
            dicReqDocToKeep[reqId][docId] = dicReqDoc[reqId][docId]
            docsToKeep.append(docId)
            i = i + 1
        reqsToKeep.append(reqId)
    docsToKeep = list(set(docsToKeep))

    #print(docsToKeep)

    #"""
    allVocab ={}
    #Construction corpus des MED
    for k in docsToKeep: #Pour chaque MED
        docTokenList = text2TokenList(dicDoc[k]) #Transforme les texte de MED en liste
        #print(docTokenList)
        for word in docTokenList: #Pour chaque mot dans la liste
            if word not in allVocab: #S'il n'y est pas déjà
                allVocab[word] = word #l'ajoute dans le dictionnaire

    allVocabListDoc = list(allVocab)
    #print("doc vocab=",allVocabListDoc)


    allVocab ={}

    #Construction corpus des PLAIN
    for k in reqsToKeep: #Pour chaque PLAIN
        docTokenList = text2TokenList(dicReq[k]) # Transforme les texte de PLAIN en liste
        #print(docTokenList)
        for word in docTokenList: # Pour chaque mot dans la liste
            if word not in allVocab: #S'il n'y est pas déjà
                allVocab[word] = word #l'ajoute dans le dictionnaire
    allVocabListReq = list(allVocab)

    from rank_bm25 import BM25Okapi

    corpusDocTokenList = []
    corpusReqTokenList = {}
    corpusDocName=[]
    corpusDicoDocName={}
    i = 0

    for k in docsToKeep: #Pour chaque MED
        docTokenList = text2TokenList(dicDoc[k]) #Transforme son corpus en liste
        corpusDocTokenList.append(docTokenList) #L'ajoute dans la liste de tous les corpus
        corpusDocName.append(k) #Ajoute le nom du MED dasn sa propre liste
        corpusDicoDocName[k] = i # Attribue un numero a chaque MED
        i = i + 1

    #print("reqs...")
    corpusReqName=[]
    corpusDicoReqName={}
    i = 0
    for k in reqsToKeep: # Pour tous les PLAIN
        reqTokenList = text2TokenList(dicReq[k]) #Transforme son corpus en liste
        corpusReqTokenList[k] = reqTokenList#créer un dictionnaire des mots pour chaque PLAIN
        corpusReqName.append(k) #Ajoute le nom du PLAIN dans sa propre liste
        corpusDicoReqName[k] = i# Attribue un numero a chaque PLAIN
        i = i + 1

    # Arrivé ici, on a :
    #- docsToKeep : une liste de tous les nom de MED
    #- allVocabListDoc : une liste de tous les mots dans les MED sans distinction sans doublons
    #- allVocabListReq : une liste de tous les mots dans les PLAIN sans distinction sans doublons
    #- corpusDocTokenList : une liste de tous les mots dans les MED sans distinction avec doublons
    #- corpusDocName : same as docsToKeep ?
    #- corpusDicoDocName : numerote chaque MED
    #- corpusReqTokenList : une liste de tous les mots dans les PLAIN sans distinction avec doublons
    #- corpusReqName : une liste de tous les nom de PLAIN
    #- corpusDicoReqName : numerote chaque PLAIN

    embeddings=modelWord2vec(corpusDocTokenList)

    print("bm25 doc indexing...")
    bm25 = BM25Okapi(embeddings) # wtf is that

    ndcgCumul=0
    corpusReqVec={}
    ndcgBM25Cumul=0
    nbReq=0
    # Calcul du score                                                  #rajout de moi
    from sklearn.metrics import ndcg_score
    for req in corpusReqTokenList: #Pour chaque mot dans le plain avec doublons
        j=0
        reqTokenList = corpusReqTokenList[req]  #liste de mots de PLAIN n°req
        doc_scores = bm25.get_scores(reqTokenList) #Computes and returns BM25 scores in relation to every item in corpus.
        #print(req)
        #print(len(doc_scores)) #nombre de mot dans chaque PLAIN ?
        trueDocs = np.zeros(len(corpusDocTokenList)) #Ressort un array de la forme de corpusDocTokenList (1779,)

        for docId in corpusDicoDocName: #pour chaque nom de MED
            if req in dicReqDocToKeep: #si PLAIN n°req est dans les recommendation
                if docId in dicReqDocToKeep[req]: # et si MED n°docId est dans les recommendation de PLAIN
                    #get position docId
                    posDocId = corpusDicoDocName[docId] #ressort le numéro du MED n°docId
                    #print(posDocId)
                    #print(req)
                    trueDocs[posDocId] = dicReqDocToKeep[req][docId] #arret de la position du numéro de MED est le score déjà donné dans reqdoc ?!
                    #print("TOKEEP=",docId)
                    #print(trueDocs)
        ndcgBM25Cumul = ndcgBM25Cumul + ndcg_score([trueDocs], [doc_scores],k=ndcgTop)
        nbReq = nbReq + 1
    ndcgBM25Cumul = ndcgBM25Cumul / nbReq
    print("ndcg bm25=",ndcgBM25Cumul)
    return ndcgBM25Cumul

In [16]:
nb_docs = 3192 #all docs
#nb_docs = 150 #for tests
cumul=run_bm25_only(0,nb_docs)

ndcgTop= 20 nbDocsToKeep= 3192
bm25 doc indexing...
ndcg bm25= 0.020026533280516328
